In [2]:
import pandas as pd
import numpy as np
import time
import datetime
import os

N_STATES = 3   # 1维世界的宽度
ACTIONS = ['buy', 'sell']     # 探索者的可用动作
EPSILON = 0.9   # 贪婪度 greedy
ALPHA = 0.1     # 学习率
GAMMA = 0.9    # 奖励递减值
MAX_EPISODES = 13   # 最大回合数
FRESH_TIME = 0.3    # 移动间隔时间
global inPortfolio
inPortfolio = False # 是否持倉中
global priceAtPurchase #損益
priceAtPurchase = 0


In [3]:
direct=os.path.abspath('../data')
filelist = os.listdir('../data')
file = filelist[-1]
dayticks = pd.read_csv(direct+'\\'+file,header=None,names=['ndatetime','nbid','nask','close','volume','deal'])
dayticks['ndatetime'] = pd.to_datetime(dayticks['ndatetime'], format='%Y-%m-%d %H:%M:%S.%f')
dayticks.sort_values(by=['ndatetime'],ascending=True)
dayticks=dayticks[(dayticks.ndatetime.dt.hour>=8) & (dayticks.ndatetime.dt.hour<15)]
dayticks.index = dayticks.ndatetime
Candledf=dayticks['close'].resample('1min',closed='right').ohlc()
tmpdf=dayticks['volume'].resample('1min').sum()
Candledf=pd.concat([Candledf,tmpdf],axis=1)
del tmpdf
tmpdf=dayticks['deal'].resample('1min').sum()
Candledf=pd.concat([Candledf,tmpdf],axis=1)
del tmpdf
Candledf['dealcumsum']=Candledf['deal'].cumsum()
del Candledf['deal']
Candledf.rename(columns={'dealcumsum':'deal'},inplace=True)
Candledf=Candledf.rename_axis('ndatetime').reset_index()
Candledf['ndatetime'] = pd.to_datetime(Candledf['ndatetime'], format='%Y-%m-%d %H:%M:%S.%f')
Candledf.sort_values(by=['ndatetime'],ascending=True)
tmpline=Candledf.close.cumsum()
Candledf['Avg_line'] = tmpline.apply(lambda x: x/(tmpline[tmpline==x].index[0]+1))
del tmpline
global CandleData
CandleData = Candledf[['Avg_line','deal','volume','close']]
print(CandleData.head())
global Total_Trade
Total_Trade = CandleData.shape[0]
Start_Run_Trade = 10
print(CandleData.at[25,'close'])


       Avg_line  deal  volume  close
0  17751.000000  -364    2351  17751
1  17744.000000  -816    1418  17737
2  17744.333333  -636     468  17745
3  17744.250000  -612     234  17744
4  17744.200000  -613     127  17744
17728


In [4]:
def build_q_table(n_states, actions):
    table = pd.DataFrame(
        np.zeros((n_states, len(actions))),     # q_table 全 0 初始
        columns=actions,    # columns 对应的是行为名称
    )
    return table

In [5]:
def choose_action(state, q_table):
    state_actions = q_table.iloc[state, :]  # 选出这个 state 的所有 action 值
    if (np.random.uniform() > EPSILON) or (state_actions.all() == 0):  # 非贪婪 or 或者这个 state 还没有探索过
        action_name = np.random.choice(ACTIONS)
    else:
        action_name = state_actions.argmax()    # 贪婪模式
    return action_name

In [6]:
def select_state(pointer):
    global CandleData
    # Find the current price of the equity
    current_price = CandleData.at[pointer,'close']
    # Find the previous price of the equity
    if pointer == 0 :
        previous_price = current_price
    else:
        previous_price = CandleData.at[pointer-1,'close']
    if current_price > previous_price:
        return 0  # Equity Appreciated
    if current_price == previous_price:
        return 1  # Equity Held Value
    if current_price < previous_price:
        return 2  # Equity Depreciated

In [18]:
# for x in range(Total_Trade):
#     result = select_state(x)
#     print(result)
q_table = build_q_table(N_STATES, ACTIONS)
trade = choose_action(3,q_table)
print(trade)

IndexError: single positional indexer is out-of-bounds

In [9]:
def determine_payoff(pointer,trade):
    global inPortfolio
    if inPortfolio :
        if trade == 0 :
            print('目前位置: {}'.format(CandleData.at[pointer,'close']))
            print('持有價格: {}'.format(priceAtPurchase))
            print('損益點數: {}'.format(CandleData.at[pointer,'close']-priceAtPurchase))
            inPortfolio = True
            return 0
        if trade == 1 :
            inPortfolio = False
            print('目前位置: {}'.format(CandleData.at[pointer,'close']))
            print('多單平倉: {}'.format(CandleData.at[pointer,'close']-priceAtPurchase))
            return CandleData.at[pointer,'close']-priceAtPurchase
        if trade == -1 :
            inPortfolio = False
            print('目前位置: {}'.format(CandleData.at[pointer,'close']))
            print('空單平倉: {}'.format(priceAtPurchase-CandleData.at[pointer,'close']))
            return priceAtPurchase-CandleData.at[pointer,'close']
    if inPortfolio is False:
        if trade == 0:
            inPortfolio = False
            return 0
        if trade == 1:
            inPortfolio = True
            print('目前位置: {}'.format(CandleData.at[pointer,'close']))
            print('多單進場: {}'.format(priceAtPurchase))
            return 0
        if trade == -1:
            inPortfolio = True
            print('目前位置: {}'.format(CandleData.at[pointer,'close']))
            print('空單進單進場: {}'.format(priceAtPurchase))
            return 0      


In [ ]:
aggregate_profit =[]
def rl():
    global Total_Trade
    q_table = build_q_table(N_STATES, ACTIONS)  # 初始 q table
    profit = 0
    for x in range(Total_Trade):
        trade = choose_action(x-1,q_table)
        print(x)
        result = determine_payoff(x, trade)
        aggregate_profit.append(result)
        # profit += result
        q_predict = q_table.iloc[select_state(x), trade]
        q_target = result + GAMMA * q_table.iloc[select_state(x), : ].max()
        q_table.iloc[select_state(x), trade] += ALPHA * (q_target - q_predict)
    profit = np.sum(aggregate_profit)
    return q_table, profit
    
    


In [ ]:
if __name__ == '__main__':
    q_table, profit = rl()
    calc_profits = profit*200
    print(q_table)
    print('總損益: {}'.format(calc_profits))